- csv と transformers ライブラリがインポートされ、transformers ライブラリから BertTokenizer と BertModel クラスを使用して BERT トークナイザとモデルがロードされています。
- 2212.csv ファイルが読み取りモードで開かれ、ファイルを読み取るために CSV リーダー オブジェクトが作成されます。
- ヘッダ行は next 関数を用いてスキップされます。
- CSV ファイルの行を繰り返し処理するために、for ループが使用されます。各行について、日付、文、ユーザ名の値が抽出される。
- 文は、BERT トークナイザーを使用して BERT 入力として符号化され、文の BERT 埋め込みは、BERT モデルを使用して計算されます。
-  文の BERT 埋め込みは、モデル出力から抽出される。
- 文の埋め込みと参照埋め込みとの間のコサイン類似度は、PyTorchのnnモジュールのCosineSimilarityクラスを使って計算される。
- 日付、文、ユーザ名、類似度の値が出力される。

In [1]:
import pandas as pd
import torch
from transformers import BertJapaneseTokenizer, BertModel
import re

MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
model = BertModel.from_pretrained(MODEL_NAME)


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
path = "/Users/iomacbookair2/Documents/lab/DEIM2023/tweet_csv/221214_ann_wed.csv"
df = pd.read_csv((path))
df.sort_values(by = 'created_at', ascending = True, inplace = True)
df = df.reset_index(drop=True)
df['created_at'] = pd.to_datetime(df['created_at'])
df = df.drop("author_id", axis=1)
df = df.drop("username", axis=1)

df
# df.to_csv("sorted.csv", index=False)

,created_at,text
0,2022-12-15 01:00:00,#乃木坂46ANN
1,2022-12-15 01:00:00,#乃木坂46ANN
2,2022-12-15 01:00:00,#乃木坂46ANN
3,2022-12-15 01:00:00,#乃木坂46ANN
4,2022-12-15 01:00:00,#乃木坂46ANN
...,...,...
13960,2022-12-15 03:03:55,おごりのカッコいい払い方って難しいけど、だからこそかっこいいのかな？ ちゃんとスポーツ観戦し...
13961,2022-12-15 03:03:56,お疲れ様でした #乃木坂46ANN
13962,2022-12-15 03:04:11,しおりさん、スタッフさんオツカレサマデシタ！！ 久々の1人喋りスポーツの話満載でしたが色んな...
13963,2022-12-15 03:04:13,12/14㈬#あちこちオードリー #乃木坂46ANN #SPYのボスの正体は佐久間宣行 #f...


In [3]:
def remove_urls_and_hashtags(text):
    # Remove URLs
    text = re.sub(r'https?://\S+', '', text)
    # Remove words starting with "#"
    text = re.sub(r'#\S+', '', text)
    text = re.sub(r'＃\S+', '', text)
    return text


df['text'] = df['text'].apply(remove_urls_and_hashtags)
df
# df.to_csv('filtered.csv', index=False)


,created_at,text
0,2022-12-15 01:00:00,
1,2022-12-15 01:00:00,
2,2022-12-15 01:00:00,
3,2022-12-15 01:00:00,
4,2022-12-15 01:00:00,
...,...,...
13960,2022-12-15 03:03:55,おごりのカッコいい払い方って難しいけど、だからこそかっこいいのかな？ ちゃんとスポーツ観戦し...
13961,2022-12-15 03:03:56,お疲れ様でした
13962,2022-12-15 03:04:11,しおりさん、スタッフさんオツカレサマデシタ！！ 久々の1人喋りスポーツの話満載でしたが色んな...
13963,2022-12-15 03:04:13,12/14㈬ 続き 3時~元テレ東P[ 📹無料映像配信も【HAKUNA Live】👉...


In [4]:
groups = df.groupby(pd.Grouper(key='created_at', freq='min'))
df_texts_by_minute = pd.DataFrame({
    "texts_by_minute": groups.apply(lambda x: x["text"].tolist())
})
df_texts_by_minute = df_texts_by_minute.reset_index()
df_texts_by_minute
# df_texts_by_minute.to_csv("texts_by_minute.csv", index=False)

,created_at,texts_by_minute
0,2022-12-15 01:00:00,"[, , , , , , , , , , , , , さあ、今夜も お願いします！ お願いし..."
1,2022-12-15 01:01:00,"[ , 遅延, , , , , , , , こんなんなんぼあってもいいですからね ,..."
2,2022-12-15 01:02:00,"[, 森高千里さんから可愛いをいただいた久保ちゃん(笑) 可愛いなんてなんぼ言われてもいいよ..."
3,2022-12-15 01:03:00,"[やかましいわ , 久保ちゃん、それ聞いちゃおしまいよ , 頑張っちゃってください, クリ..."
4,2022-12-15 01:04:00,"[あの森高千里さんから「かわいい」頂きましたｗ , クリスマスツリーとシュトーレンを買う..."
...,...,...
120,2022-12-15 03:00:00,[今週もお疲れ様でした！ かっこつけようとして、失敗してしまったのがくぼちゃんらしいです笑 ...
121,2022-12-15 03:01:00,[『乃木坂46のオールナイトニッポン』、卒業を発表している齋藤飛鳥の生出演が決定！ パーソ...
122,2022-12-15 03:02:00,"[またねー , しおりんお疲れ様でした〜！ 来週も絶対聴く！👂 またね〜(´▽`)ﾉ ..."
123,2022-12-15 03:03:00,"[敗者復活でやるような場所じゃない , 久保ちゃんお疲れ様でした🎵 ソロはひさびさでしたね ..."


In [5]:
def sentence_to_vector(model, tokenizer, sentence):
    tokens = tokenizer(sentence, add_special_tokens=True)["input_ids"]
    input = torch.tensor(tokens).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input, output_hidden_states=True)
        last_hidden_state = outputs[0][:, 0, :]
        averaged_hidden_state = last_hidden_state.mean(dim=0).unsqueeze(0)
    return averaged_hidden_state


def calc_similarity(sentence1, sentence2):
    print("{}\n{}".format(sentence1, sentence2))

    sentence_vector1 = sentence_to_vector(model, tokenizer, sentence1)
    sentence_vector2 = sentence_to_vector(model, tokenizer, sentence2)

    # Reshape the tensors to 1D
    sentence_vector1 = sentence_vector1.reshape(-1)
    sentence_vector2 = sentence_vector2.reshape(-1)

    similarity = float(torch.nn.functional.cosine_similarity(
        sentence_vector1, sentence_vector2, dim=0).detach().numpy().copy())
    print("Similarity:", similarity)

    return similarity


def calc_average_similarity(sentences):
    similarities = []
    for i in range(1, len(sentences)):
        similarity = calc_similarity(sentences[0], sentences[i])
        similarities.append(similarity)
    average_similarity = sum(similarities) / len(similarities)
    # print("Similarities:", similarities)
    print("Average similarity:", average_similarity)

    return average_similarity

In [6]:
df_texts_by_minute["average_similarity"] = df_texts_by_minute["texts_by_minute"].apply(
    calc_average_similarity)
df_texts_by_minute




Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209

さあ、今夜も お願いします！ お願いします！ お願いします！     
Similarity: 0.6659478545188904


Similarity: 0.9999997615814209

 
Similarity: 0.9999997615814209


Similarity: 0.9999997615814209

乃木坂46のオールナイトニッポン | ニッポン放送 | 2022/12/14/水  25:00-27:00     
Similarity: 0.7102256417274475


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0.9999997615814209


Similarity: 0

,created_at,texts_by_minute,average_similarity
0,2022-12-15 01:00:00,"[, , , , , , , , , , , , , さあ、今夜も お願いします！ お願いし...",0.977954
1,2022-12-15 01:01:00,"[ , 遅延, , , , , , , , こんなんなんぼあってもいいですからね ,...",0.864120
2,2022-12-15 01:02:00,"[, 森高千里さんから可愛いをいただいた久保ちゃん(笑) 可愛いなんてなんぼ言われてもいいよ...",0.738300
3,2022-12-15 01:03:00,"[やかましいわ , 久保ちゃん、それ聞いちゃおしまいよ , 頑張っちゃってください, クリ...",0.669687
4,2022-12-15 01:04:00,"[あの森高千里さんから「かわいい」頂きましたｗ , クリスマスツリーとシュトーレンを買う...",0.726158
...,...,...,...
120,2022-12-15 03:00:00,[今週もお疲れ様でした！ かっこつけようとして、失敗してしまったのがくぼちゃんらしいです笑 ...,0.814245
121,2022-12-15 03:01:00,[『乃木坂46のオールナイトニッポン』、卒業を発表している齋藤飛鳥の生出演が決定！ パーソ...,0.742052
122,2022-12-15 03:02:00,"[またねー , しおりんお疲れ様でした〜！ 来週も絶対聴く！👂 またね〜(´▽`)ﾉ ...",0.784267
123,2022-12-15 03:03:00,"[敗者復活でやるような場所じゃない , 久保ちゃんお疲れ様でした🎵 ソロはひさびさでしたね ...",0.683805


In [7]:
df_texts_by_minute.to_csv('cosine_similarity_221214_ann_wed.csv', index=False)